In [11]:
import numpy as np
import scipy
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.svm import SVC
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
def cross_validation(data,labels, penalty, kernel, sigma = None):
    #split into training set and test set
    l = len(data)
    
    kf = KFold(n_splits=2)

    train_index, test_index in kf.split(data)


In [7]:
 d = np.random.choice(2, 12)

In [10]:
e= np.array(range(12))
e[d]

array([1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1])